# Session 14: Face Detection

We extend deep learning models to deal with a specific type of object detection:
the localization and identification of faces.

In [2]:
%pylab inline

import numpy as np
import scipy as sp
import pandas as pd
import sklearn
from sklearn import linear_model
import urllib

import os
from os.path import join

Populating the interactive namespace from numpy and matplotlib


In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.rcParams["figure.figsize"] = (8,8)

In [4]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

## dlib: A frustrating library for face detection

We need a few new libraries today. You should be able to 
install these with:

    pip install cmake
    pip install dlib
    pip install face_recognition
    
Load in the face_recognition library with the following:

In [ ]:
import face_recognition as fr

## Bewitched

Let's load and look at the the bewitch corpus. It contains still images
from two episodes of the sitcom Bewitched.

In [ ]:
df = pd.read_csv(join("..", "data", "bewitched.csv"))
df.head()

If you are struggling with installing these, we are happy to assist. You'll be able to follow
along with keras, but will not be able to apply the techniques you learned today to new datasets
without it.

## Face detection

As an example of what this new corpus looks like, here is an image of Darrin and Samantha
in their living room at the start of the episode "Witches and Warlocks Are my Favorite
Things".

In [ ]:
img_path = join('..', 'images', 'bewitched', df.filename[200])
img = imread(img_path)
plt.imshow(img)

There are two faces in this image, which we can detect using the face_recognition
algorithm.

In [ ]:
faces = fr.face_locations(img, 1, model="cnn")
    
print(faces)

The output indicates that two faces are detected, and the numbers gives the
coordinates for the faces known as *bounding boxes*. We can plot them in Python
with the following snippet of code.

In [ ]:
fig,ax = plt.subplots(1,1)
plt.imshow(img)
n, m, d = img.shape
for face in faces:
    rect = plt.Rectangle((face[3], face[0]), face[2] - face[0], face[1] - face[3],
                         edgecolor='orange', linewidth=2, facecolor='none')
    ax.add_patch(rect)

And, as hoped, the detected faces line up with the two characters in the frame.

## Face identification

In addition to detected *where* a face is, we also want to determine *who* the face
belongs to. In order to do this, we again make use of a pre-trained neural network
that returns a sequence of numbers. Just as with image similarity, we assume that
faces of the same person are identified with similar sequences of numbers.

To illustrate how this works, lets take a set of four faces from Bewitched. The first
two are of the same character (Samantha) but the second two are of of Larry and Darrin,
respectively.

In [ ]:
plt.figure(figsize=(14, 14))

for id, index in enumerate([145, 300, 420, 707]):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.subplot(1, 4, id + 1)

    img_path = join('..', 'images', 'bewitched', df.filename[index])
    img = imread(img_path)
    plt.imshow(img)
    plt.axis("off")

We can compute the 128-dimension number associated with each face using the function
`fr.face_encodings` applied to each face.

In [ ]:
embed = []
for id, index in enumerate([300, 145, 420, 707]):
    img_path = join('..', 'images', 'bewitched', df.filename[index])
    img = imread(img_path)

    f = fr.face_locations(img, 1, model="cnn")
    e = fr.face_encodings(img, known_face_locations=[f[0]])
    embed.append(e[0])
    
embed = np.array(embed)

Using the first image of Samantha as a baseline, look at how close each of the other three
images are to it.

In [ ]:
np.sum((embed - embed[0, :])**2, 1)

The other image of Samantha is just 0.202 away but the images of the two male characters
are 0.709 and 0.710 away. Using a cut-off (around 0.35 works well), we can identify images
of Samantha with a reasonably high accuracy.

### Faces at scale

Finally, let's apply our face detection algorithm of the entire corpus of Bewitched
iamges. The face detect is fairly slow (it took about 30 minutes on a small MacBook
to do all of the images), so we recommend you set `process_new` to `False` and load
the faces in from the save file.

In [6]:
process_new = False

if process_new:
    embed = []; output = []
    corpus = pd.read_csv(join("meta", cn + ".csv"))
    for index, row in corpus.iterrows():
        img_path = join('images', cn, row['filename'])
        img = sp.misc.imread(img_path)
        faces = fr.face_locations(img, 1, model="cnn")
        output.append(faces)
        enc = fr.face_encodings(img, known_face_locations=faces)
        embed.append(enc)
else:
    faces = np.load(join("..", "data", "bewitched_faces.npy"))
    embed = np.load(join("..", "data", "bewitched_embed.npy"))

We will compare each of the embeddings to first image used in Step 13, storing the
distance in the array `samantha_dist`.

In [ ]:
samantha = embed[145][0]

samantha_dist = np.ones(len(df))
for item in range(embed.shape[0]):
    for em in embed[item]:
        samantha_dist[item] = np.sum((samantha - em)**2)

Using a cut-off of 0.35, how many frames contain an image of Samantha?

In [ ]:
np.sum(samantha_dist < 0.35)

It looks like a total of 119 images show a definitive image of Samantha. Using a similar
block of code to the similarity metrics used throughout these notes, what images contain
the most similar Samantha faces to our prototype image?

In [ ]:
plt.figure(figsize=(14, 24))

sam_index = np.argsort(samantha_dist).tolist()
for ind, i in enumerate(sam_index[:24]):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.subplot(8, 3, ind + 1)

    img_path = join('..', 'images', 'bewitched', df.filename[i])
    img = imread(img_path)
    plt.imshow(img)
    plt.axis("off")

Like our prototype, almost all of these indicates a relatively large view of Samantha looking
straight at the camera.

**Now, use image 707 as a baseline for Darrin and compute a darrin_distance metric for
each image**.

In [ ]:
darrin = embed[707][0]

darrin_dist = np.ones(len(df))
for item in range(embed.shape[0]):
    for em in embed[item]:
        darrin_dist[item] = np.sum((darrin - em)**2)

**Then, display the top 24 images of Darrin.**

In [ ]:
plt.figure(figsize=(14, 24))

darrin_index = np.argsort(darrin_dist).tolist()
for ind, i in enumerate(darrin_index[:24]):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.subplot(8, 3, ind + 1)

    img_path = join('..', 'images', 'bewitched', df.filename[i])
    img = imread(img_path)
    plt.imshow(img)
    plt.axis("off")

**How do these compare to those of Samantha?**